In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
df = pd.read_csv('../Data/glassdoor_jobs_US.csv')
df.head()

,company,job_title,company_rating,job_description,location,salary_avg_estimate,salary_estimate_payperiod,company_size,company_founded,employment_type,industry,sector,revenue,career_opportunities_rating,comp_and_benefits_rating,culture_and_values_rating,senior_management_rating,work_life_balance_rating
0,Reliance Telephone Systems,Data And Telephone Technician,NaN,Data and Telephone Technician\nDuties:\n- Inst...,"Grand Forks, ND",$20.00,/hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,One of our clients,Head of Engineering - Data Centre,NaN,Essential Functions:\nWork in partnership with...,"Pryor, OK",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Big Sky Managed Care,Certified Pharmacy Technician (CPhT) Data entr...,2.4,Our locally owned pharmacy group has an immedi...,"Great Falls, MT",$21.00,/hr (est.),51 to 200 Employees,2015,Company - Private,Pharmaceutical,Pharmaceutical & Biotechnology,$5 to $25 million (USD),2.4,2.6,2.1,2.6,2.6
3,HUNTER OIL AND PROPANE,Data Entry 2-3 years experience,NaN,```Duties```\n- Prepare and analyze financial ...,"Sanford, NC","$45,760",/yr (est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Waabigwan Mashkiki,Data and Finance Administrator,NaN,"Waabigwan Mashkiki, LLC\nPosition Description\...","Mahnomen, MN",$22.50,/hr (est.),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   company                      780 non-null    object 
 1   job_title                    780 non-null    object 
 2   company_rating               607 non-null    float64
 3   job_description              763 non-null    object 
 4   location                     780 non-null    object 
 5   salary_avg_estimate          700 non-null    object 
 6   salary_estimate_payperiod    700 non-null    object 
 7   company_size                 612 non-null    object 
 8   company_founded              612 non-null    object 
 9   employment_type              612 non-null    object 
 10  industry                     612 non-null    object 
 11  sector                       612 non-null    object 
 12  revenue                      612 non-null    object 
 13  career_opportunities

In [6]:
df.isna().sum()

company                          0
job_title                        0
company_rating                 173
job_description                 17
location                         0
salary_avg_estimate             80
salary_estimate_payperiod       80
company_size                   168
company_founded                168
employment_type                168
industry                       168
sector                         168
revenue                        168
career_opportunities_rating    190
comp_and_benefits_rating       190
culture_and_values_rating      190
senior_management_rating       190
work_life_balance_rating       190
dtype: int64

## text prosessing to extract skills, qualifications, etc.

In [15]:
# remove postings with no description
df_clean = df[~df['job_description'].isna()]

In [29]:
# make titles and descriptions lowercase
df_clean['job_title'] = df_clean['job_title'].str.lower()
df_clean['job_description'] = df_clean['job_description'].str.lower()

C:\Users\Shane\AppData\Local\Temp\ipykernel_17848\552748824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['job_title'] = df_clean['job_title'].str.lower()
C:\Users\Shane\AppData\Local\Temp\ipykernel_17848\552748824.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['job_description'] = df_clean['job_description'].str.lower()


In [30]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 763 entries, 0 to 779
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   company                      763 non-null    object 
 1   job_title                    763 non-null    object 
 2   company_rating               591 non-null    float64
 3   job_description              763 non-null    object 
 4   location                     763 non-null    object 
 5   salary_avg_estimate          700 non-null    object 
 6   salary_estimate_payperiod    700 non-null    object 
 7   company_size                 611 non-null    object 
 8   company_founded              611 non-null    object 
 9   employment_type              611 non-null    object 
 10  industry                     611 non-null    object 
 11  sector                       611 non-null    object 
 12  revenue                      611 non-null    object 
 13  career_opportunities

In [113]:
# filter out jobs that aren't data analytics / data science related based on title
positive_filter = {'science', 'scientist', 'analyst', 'analytics', 'machine', 'learning', 'programmer', 'developer', 'engineer'}
negative_filter = {'entry'} # filter out data entry jobs

In [114]:
def filter_words(text):
    words = set(text.split())
    return bool(positive_filter & words) and not bool(negative_filter & words)

In [119]:
df_filtered = df_clean[df_clean['job_title'].apply(filter_words)]
df_filtered.head()

,company,job_title,company_rating,job_description,location,salary_avg_estimate,salary_estimate_payperiod,company_size,company_founded,employment_type,industry,sector,revenue,career_opportunities_rating,comp_and_benefits_rating,culture_and_values_rating,senior_management_rating,work_life_balance_rating
14,Oak Creek Farms - Fort Independence Indian Com...,cannabis data analytics and marketing director,3.1,the director of analytics & marketing oak cree...,"Independence, CA",$29.50,/hr (est.),1 to 50 Employees,--,Government,--,--,Unknown / Non-Applicable,3.1,3.4,2.9,2.9,2.5
31,Charter Manufacturing,business intelligence and analytics lead - dat...,4.2,charter manufacturing is a fourth-generation f...,"Saukville, WI","$97,836",/yr (est.),1001 to 5000 Employees,1936,Company - Private,Metal & Mineral Manufacturing,Manufacturing,$1 to $5 billion (USD),4.4,4.4,4.3,4.2,3.8
34,Green Bay Converting,manufacturing business/data analyst,3.4,overview:\nthe manufacturing business/data ana...,"De Pere, WI","$72,500",/yr (est.),51 to 200 Employees,1999,Company - Private,Machinery Manufacturing,Manufacturing,Unknown / Non-Applicable,3.2,3.4,3.2,3.1,3.9
39,Phoenix Lithographing Corporation,direct mail data processing programmer - comme...,3.0,"phoenix lithographing corporation, a commercia...","Philadelphia, PA","$72,807",/yr (est.),51 to 200 Employees,--,Company - Public,Consumer Product Manufacturing,Manufacturing,$25 to $100 million (USD),2.8,2.8,2.8,2.8,2.8
41,Pete's RV Center,sr. data analyst,4.0,pete's rv is looking for a detail-oriented hyb...,"South Burlington, VT","$85,000",/yr (est.),1 to 50 Employees,--,Company - Private,--,--,Unknown / Non-Applicable,NaN,NaN,NaN,NaN,NaN


In [118]:
df_filtered.job_title.tolist()

['cannabis data analytics and marketing director',
 'business intelligence and analytics lead - data engineer',
 'manufacturing business/data analyst',
 'direct mail data processing programmer - commercial print / direct mail',
 'sr. data analyst',
 'senior data analyst',
 'health information technician- data analytics & medical records',
 'data center operations engineer',
 'software developer - ai trainer (contract)',
 'informatica developer with data movement',
 'operations data analyst',
 'senior data analyst/adobe campaign developer',
 'revenue & data analyst',
 'data processing analyst',
 'data engineer ii - qlik analytics lead',
 'data warehouse engineer',
 'business analyst',
 'army aviation data analyst',
 'data engineer',
 'data analytics engineer',
 'sentinel principal / sr. principal data visualization engineer – 10335',
 'system / business analyst (specifically, vista by viewpoint)',
 'research data analyst ii | agoro lab',
 'business intelligence and analytics lead - data